In [1]:
# Import packages

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
%matplotlib inline
from sklearn import preprocessing, tree, metrics

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import numpy as np
import math
import scipy
from scipy import ndimage
from tensorflow.python.framework import ops

In [2]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    
    m = X.shape[0]
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [3]:
labeled_images = pd.read_csv('./data/train.csv')
images = labeled_images.iloc[:, 1:]
labels = labeled_images.iloc[:, :1]
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, train_size=0.7, random_state=0)

In [4]:
test_images[test_images>0] = 1
train_images[train_images>0] = 1

# train_images = shape_image(train_images)
# test_images = shape_image(test_images)

train_images = np.array(train_images)
test_images = np.array(test_images)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

train_labels = convert_to_one_hot(train_labels, 10).T
test_labels = convert_to_one_hot(test_labels, 10).T

X_train = train_images.astype("float")
Y_train = train_labels.astype("float")
X_test = test_images.astype("float")
Y_test = test_labels.astype("float")

In [7]:
x = tf.placeholder(tf.float32, [None, 784], name='x')
W1 = tf.Variable(tf.zeros([784, 64]))
b1 = tf.Variable(tf.zeros([64]))

z1 = tf.nn.relu(tf.matmul(x, W1) + b1)

W2 = tf.Variable(tf.zeros([64, 10]))
b2 = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(z1, W2) + b2)
y_ = tf.placeholder(tf.float32, [None, 10], name='y_')

cost = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cost)
sess = tf.Session()

init = tf.global_variables_initializer()
# init = tf.initialize_all_variables()
sess.run(init)

In [8]:
minibatch_size = 64
seed = 3

m = X_train.shape[0]
minibatch_cost = 0.
num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set

for i in range(3):
    seed = seed + 1
    minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

    for mb_x, mb_y in minibatches:
    #     print(mb_x.shape, mb_y.shape)
    #     print(mb_x, mb_y)

        sess.run(train_step, feed_dict={x: mb_x, y_: mb_y})
    #     minibatch_cost += temp_cost / num_minibatches
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        acc = sess.run(accuracy, feed_dict={x: mb_x, y_:mb_y})
    print("epoc " + str(i) + ":" + str(acc))

print(sess.run(accuracy, feed_dict={x:X_test, y_:Y_test}))
saver = tf.train.Saver()
model_path = "./checkpoints/model1.ckpt"
save_path = saver.save(sess, model_path)

epoc 0:0.173913
epoc 1:0.0869565


KeyboardInterrupt: 